In [ ]:
from __future__ import annotations

from pathlib import Path
from typing import Literal, cast

type Position = tuple[int, int]
type Direction = Literal["<", ">", "^", "v"]

In [ ]:
def position_in_direction(position: Position, direction: Direction) -> Position:
    """Return the position in the given direction from the given position."""
    row, col = position
    dirmap = {"<": (0, -1), ">": (0, 1), "^": (-1, 0), "v": (1, 0)}
    drow, dcol = dirmap[direction]
    return (row + drow, col + dcol)

In [ ]:
class Box:
    """A box in the warehouse."""

    def __init__(self, position: Position, part2: bool = False) -> None:
        """Initialize a box at the given position."""
        self.position = position
        self.part2 = part2

    def move(self, direction: Direction) -> None:
        """Move the box in the given direction."""
        self.position = position_in_direction(self.position, direction)

    def is_blocked(self, direction: Direction, walls: set[Position]) -> bool:
        """Check if the box is blocked in the given direction by walls."""
        return any(
            position_in_direction(point, direction) in walls
            for point in self.coordinates
        )

    def touching_other_boxes(
        self, direction: Direction, others: list[Box]
    ) -> list[Box]:
        """Return the boxes touching this box in the given direction."""
        touching = set()
        for other in others:
            for point in self.coordinates:
                if position_in_direction(point, direction) in other.coordinates:
                    touching.add(other)
        return list(touching)

    @property
    def gps_coordinates(self) -> int:
        """Return the GPS coordinates of the box."""
        row, col = self.position
        return 100 * row + col

    @property
    def coordinates(self) -> set[Position]:
        """Return all the coordinates of the box."""
        if not self.part2:
            return {self.position}
        row, col = self.position
        return {(row, col), (row, col + 1)}

In [ ]:
class Robot:
    """A robot that can move around the map and push boxes."""

    def __init__(
        self, position: Position, walls: set[Position], boxes: list[Box]
    ) -> None:
        """Initialize the robot at the given position."""
        self.position = position
        self.walls = walls
        self.boxes = boxes

    def move(self, direction: Direction) -> None:
        """Move the robot in the given direction."""
        move_boxes = self.can_move_in_direction(direction)
        if move_boxes is not None:
            for box in reversed(move_boxes):
                box.move(direction)
            self.position = position_in_direction(self.position, direction)

    def can_move_in_direction(self, direction: Direction) -> None | list[Box]:
        """Can we move in this direction? What boxes would be pushed?"""
        # If it is possible to move in the given direction, return the positions of the
        # boxes that would be pushed. Otherwise return None."""
        new_position = position_in_direction(self.position, direction)
        move_boxes = self.find_colliding_boxes(direction)
        if new_position in self.walls:
            return None
        if any(box.is_blocked(direction, self.walls) for box in move_boxes):
            return None
        return move_boxes

    def find_colliding_boxes(self, direction: Direction) -> list[Box]:
        """Find all boxes that would collide if we moved in the given direction."""
        colliding: list[Box] = []
        queue: list[Box] = []
        new_position = position_in_direction(self.position, direction)
        for box in self.boxes:
            if new_position in box.coordinates:
                colliding.append(box)
                queue.append(box)
        while queue:
            box = queue.pop()
            if others := box.touching_other_boxes(direction, self.boxes):
                for other in others:
                    if other in colliding:
                        continue
                    colliding.append(other)
                    queue.append(other)
        return colliding

In [ ]:
def read_input() -> tuple[Robot, list[Direction]]:
    """Read the input for part 1."""
    with Path("day15_input.txt").open() as file:
        walls: set[Position] = set()
        boxes: list[Box] = []
        directions: list[Direction] = []
        for row, line in enumerate(file):
            if line.startswith("#"):
                for col, char in enumerate(line):
                    if char == "#":
                        walls.add((row, col))
                    elif char == "O":
                        boxes.append(Box((row, col)))
                    elif char == "@":
                        robot_pos = (row, col)
            elif line[0] in "<>^v":
                directions.extend(cast(list[Direction], list(line.strip())))

    return Robot(robot_pos, walls, boxes), directions

# Part 1


In [ ]:
robot, directions = read_input()

for direction in directions:
    robot.move(direction)

sum(box.gps_coordinates for box in robot.boxes)

# Part 2


In [ ]:
def read_input_part2() -> tuple[Robot, list[Direction]]:
    """Read the input for part 2."""
    walls: set[Position] = set()
    boxes: list[Box] = []
    directions: list[Direction] = []
    with Path("day15_input.txt").open() as file:
        for row, line in enumerate(file):
            if line.startswith("#"):
                for col, char in enumerate(line):
                    if char == "#":
                        walls.add((row, col * 2))
                        walls.add((row, col * 2 + 1))
                    elif char == "O":
                        boxes.append(Box((row, col * 2), part2=True))
                    elif char == "@":
                        robot_pos = (row, col * 2)
            elif line[0] in "<>^v":
                directions.extend(cast(list[Direction], list(line.strip())))

    return Robot(robot_pos, walls, boxes), directions

In [ ]:
robot, directions = read_input_part2()

for direction in directions:
    robot.move(direction)

sum(box.gps_coordinates for box in robot.boxes)

# Debugging


In [ ]:
def print_grid(robot: Robot) -> None:
    """Print the grid with the robot and boxes."""
    for row in range(10):
        for col in range(20):
            if (row, col) in robot.walls:
                print("#", end="")
            elif (row, col) == robot.position:
                print("@", end="")
            elif any((row, col) in box.coordinates for box in robot.boxes):
                print("O", end="")
            else:
                print(".", end="")
        print("")
    print("")

In [ ]:
robot, directions = read_input_part2()
print_grid(robot)

In [ ]:
robot, directions = read_input_part2()
print_grid(robot)
for direction in directions:
    print("Move in direction", direction)
    robot.move(direction)
    print_grid(robot)